# Generating pitch tracks for the Guše visualizer

In [1]:
import json
import os
import numpy as np
import os
import matplotlib.pyplot as plt

In [37]:
folderName = '../files/pitchTracks/'
mbid = '66a75b20-87a4-4ee8-b1b5-5d1d96749db5'
pitchTrackExtension = '.f0_fil.csv'
fileName = mbid + pitchTrackExtension

dur = 56
shahed = 227.61

filePath = os.path.join(folderName, fileName)
print(filePath)

../files/pitchTracks/66a75b20-87a4-4ee8-b1b5-5d1d96749db5.f0_fil.csv


### From Hz to cents

In [38]:
def h2c(t, h):
    return np.rint(1200 * np.log2(h/t))

In [39]:
# Load csv
pitchTrackHz = np.genfromtxt(filePath, delimiter=",")
pitchTrackHz

array([[  0.34 , 176.271],
       [  0.35 , 178.618],
       [  0.36 , 181.497],
       ...,
       [ 55.65 , 207.956],
       [ 55.66 , 205.545],
       [ 55.68 , 196.843]])

In [40]:
maxCent = h2c(shahed, np.max(pitchTrackHz[:,1]))
print("Max:", maxCent)

minCent = h2c(shahed, np.min(pitchTrackHz[:,1]))
print("Min:", minCent)

Max: 3262.0
Min: -972.0


In [41]:
maxCent = 500
# minCent = -722

minCut = True

In [42]:
# Prepare dic
pitchTrackDic = {}

for frame in pitchTrackHz:
    pitchTrackDic["{:.2f}".format(frame[0])] = frame[1]

In [43]:
# Find duration

if pitchTrackHz[-1,0] > dur:
    dur = pitchTrackHz[-1,0]

dur

56

In [9]:
# Cent pitch track

pitchTrackCents = {}

for i in range(dur * 100):
    k = "{:.2f}".format(i/100)
    if k not in pitchTrackDic.keys():
        pitchTrackCents[k] = "s"
    else:
        c = h2c(shahed, pitchTrackDic[k])
        if minCut:
            if c >= minCent and c <= maxCent:
                pitchTrackCents[k] = c
        else:
            pitchTrackCents[k] = c

pitchTrackCents

{'0.00': 's',
 '0.01': 's',
 '0.02': 's',
 '0.03': 's',
 '0.04': 's',
 '0.05': 's',
 '0.06': 's',
 '0.07': 's',
 '0.08': 's',
 '0.09': 's',
 '0.10': 's',
 '0.11': 's',
 '0.12': 's',
 '0.13': 's',
 '0.14': 's',
 '0.15': 's',
 '0.16': 's',
 '0.17': 's',
 '0.18': 's',
 '0.19': 's',
 '0.20': 's',
 '0.21': 's',
 '0.22': 's',
 '0.23': 's',
 '0.24': 's',
 '0.25': 's',
 '0.26': 's',
 '0.27': 's',
 '0.28': 's',
 '0.29': 's',
 '0.30': 's',
 '0.31': 's',
 '0.32': 's',
 '0.33': 's',
 '0.34': -443.0,
 '0.35': -420.0,
 '0.36': -392.0,
 '0.37': -351.0,
 '0.38': -296.0,
 '0.39': -263.0,
 '0.40': -249.0,
 '0.41': -241.0,
 '0.42': -233.0,
 '0.43': -224.0,
 '0.44': -217.0,
 '0.45': -211.0,
 '0.46': -197.0,
 '0.47': -187.0,
 '0.48': -180.0,
 '0.49': -173.0,
 '0.50': -170.0,
 '0.51': -170.0,
 '0.52': -170.0,
 '0.53': -168.0,
 '0.54': -165.0,
 '0.55': -159.0,
 '0.56': -156.0,
 '0.57': -159.0,
 '0.58': -169.0,
 '0.59': 's',
 '0.60': 's',
 '0.61': 's',
 '0.62': 's',
 '0.63': 's',
 '0.64': 's',
 '0.65': 's',
 

In [44]:
with open(folderName + mbid + "_pitchTrack.json", 'w') as f:
    json.dump(pitchTrackCents, f)

In [45]:
x = 0
y = 0
for k in pitchTrackCents.keys():
    if pitchTrackCents[k] != 's' and pitchTrackCents[k] > x:
        x = pitchTrackCents[k]
    if pitchTrackCents[k] != 's' and pitchTrackCents[k] < y:
        y = pitchTrackCents[k]
print(y, x)

-972.0 3262.0


### Pitch histogram

In [32]:
import intonation
import numpy as np
import json
import os

In [33]:
def pitch2cents(pitch, ref):
    '''
    Converts a pitch track in Hz to cents given a reference pitch
    
    Args:
        pitch (np.array): a numpy array with two columns, first for time,
                          second for pitch
        ref (float): the reference pitch
    
    Returns:
        cents (np.array) a numpy array with one column of pitch in cents
    '''

    return 1200 * np.log2(pitch[:,1] / float(ref))

In [34]:
# folderName = './'
# mbid = '7d88c651-5dea-48e5-b21a-ac964cff5cad'
# pitchTrackExtension = '.f0_fil.csv'

# shahed = 332.67

In [35]:
pitchTrack = np.genfromtxt(os.path.join(folderName, mbid + pitchTrackExtension), delimiter=',')
centsTrack = pitch2cents(pitchTrack, shahed)
filteredTrack = centsTrack[np.logical_and(centsTrack >= minCent, centsTrack <= maxCent)]
pitch_obj = intonation.Pitch(np.arange(len(filteredTrack)), filteredTrack)
# pitch_obj = intonation.Pitch(np.arange(len(centsTrack)), centsTrack)
rec_obj = intonation.Recording(pitch_obj)
rec_obj.compute_hist()

print(rec_obj.histogram.x)
print(rec_obj.histogram.y)

[-4757.00755673 -4756.00751481 -4755.0074729  ...  1021.23463313
  1022.23467505  1023.23471696]
[1.66797154e-05 1.63444647e-05 1.56940424e-05 ... 4.92429933e-05
 4.82641159e-05 4.77561856e-05]


In [20]:
histDic = {}

for i in range(rec_obj.histogram.x.size):
    # pos = '{:.2f}'.format(rec_obj.histogram.x[i])
    x = round(rec_obj.histogram.x[i], 2)
    y = round(rec_obj.histogram.y[i] * 10000, 2)
    histDic[x] = y

histMin = np.min(list(histDic.values()))
histMax = np.max(list(histDic.values()))

print(histMin, histMax)

histList = []
for k, v in histDic.items():
    histList.append([k,v])

hist2json = {'min': histMin, 'max': histMax, 'hist': histList}
hist2json

0.0 81.14


{'min': 0.0,
 'max': 81.14,
 'hist': [[-970.56, 0.31],
  [-969.56, 0.3],
  [-968.56, 0.3],
  [-967.56, 0.29],
  [-966.56, 0.28],
  [-965.56, 0.27],
  [-964.56, 0.26],
  [-963.56, 0.26],
  [-962.56, 0.25],
  [-961.56, 0.25],
  [-960.56, 0.25],
  [-959.56, 0.25],
  [-958.56, 0.26],
  [-957.56, 0.27],
  [-956.56, 0.28],
  [-955.56, 0.29],
  [-954.56, 0.3],
  [-953.56, 0.31],
  [-952.56, 0.32],
  [-951.56, 0.33],
  [-950.56, 0.34],
  [-949.56, 0.34],
  [-948.56, 0.33],
  [-947.56, 0.33],
  [-946.56, 0.32],
  [-945.56, 0.3],
  [-944.56, 0.28],
  [-943.56, 0.26],
  [-942.56, 0.24],
  [-941.55, 0.21],
  [-940.55, 0.19],
  [-939.55, 0.16],
  [-938.55, 0.14],
  [-937.55, 0.12],
  [-936.55, 0.1],
  [-935.55, 0.08],
  [-934.55, 0.07],
  [-933.55, 0.06],
  [-932.55, 0.05],
  [-931.55, 0.04],
  [-930.55, 0.04],
  [-929.55, 0.05],
  [-928.55, 0.05],
  [-927.55, 0.06],
  [-926.55, 0.06],
  [-925.55, 0.07],
  [-924.55, 0.08],
  [-923.55, 0.1],
  [-922.55, 0.11],
  [-921.55, 0.12],
  [-920.55, 0.13],
 

In [36]:
pitchHistogramFolder = '../files/pitchHistograms'
pitchHistogramExtension = '_pitchHistogram.json'

json.dumps(hist2json)

with open(os.path.join(pitchHistogramFolder, mbid + pitchHistogramExtension), 'w') as f:
    json.dump(hist2json, f)